In [1]:
import numpy as np
import os
import math
import cv2
import matplotlib.pyplot as plt
import dlib
from numpy.linalg import svd, inv
from collections import namedtuple
from scipy.spatial import ConvexHull
#code from here: https://bitbucket.org/william_rusnack/minimumboundingbox/src/master/MinimumBoundingBox.py

np.set_printoptions(suppress=True)
%matplotlib inline

In [2]:
def computeHomography(pts1, pts2):
    '''
    Compute homography that maps from pts1 to pts2 using least squares solver
     
    Input: pts1 and pts2 are 3xN matrices for N points in homogeneous
    coordinates. 
    
    Output: H is a 3x3 matrix, such that pts2~=H*pts1
    '''    
    # Normalization ops
    T_1   = np.eye(3) 
    T_1[:,-1] = np.array([-np.mean(pts1[0,:]), -np.mean(pts1[1,:]), 1])
    S_1 = np.diag(np.array([1/np.std(pts1[0,:]), 1/np.std(pts1[1,:]), 1])) 

    T_2 = np.eye(3) 
    T_2[:,-1] = np.array([-np.mean(pts2[0,:]), -np.mean(pts2[1,:]), 1])
    S_2 = np.diag(np.array([1/np.std(pts2[0,:]), 1/np.std(pts2[1,:]), 1]))

    # Apply normalization
    pts1, pts2 = S_1 @ (T_1 @ pts1), S_2 @ (T_2 @ pts2)
    
    # Create A matrix
    A = np.zeros((pts1.shape[1]*2, 9))

    for i in range(pts1.shape[1]):   
        A[(2*i),:] = np.asarray([-pts1[0,i], -pts1[1,i], -1, 0, 0, 0, pts1[0,i]*pts2[0,i], pts1[1,i]*pts2[0,i], pts2[0,i]])
        A[(2*i)+1,:] = np.asarray([0,0,0, -pts1[0,i], -pts1[1,i],-1, pts1[0,i]*pts2[1,i], pts1[1,i]*pts2[1,i], pts2[1,i]])
    
    # SVD
    u,s,V = np.linalg.svd(A, full_matrices=True, compute_uv=True)
    H = V[-1,:].reshape((3, 3))
    H_ = (np.linalg.inv(T_2) @ (np.linalg.inv(S_2) @ ((H@S_1) @ T_1)))/H[2,2]
    
    return H_

In [3]:
def unit_v(p0, p1):
    a = (p0[0] - p1[0]); b = (p0[1] - p1[1])
    euler_dist = math.sqrt(a**2 + b**2)
    return a/euler_dist, b/euler_dist

def orth_v(vec):
    return -1*vec[1], vec[0]

def bounding_area(index, hull):
    unit_vector_p = unit_v(hull[index], hull[index+1])
    unit_vector_o = orth_v(unit_vector_p)

    dis_p = tuple(np.dot(unit_vector_p, pt) for pt in hull)
    dis_o = tuple(np.dot(unit_vector_o, pt) for pt in hull)

    min_p = min(dis_p)
    min_o = min(dis_o)
    len_p = max(dis_p) - min_p
    len_o = max(dis_o) - min_o

    return {'area': len_p * len_o,
            'length_parallel': len_p,
            'length_orthogonal': len_o,
            'rectangle_center': (min_p + len_p / 2, min_o + len_o / 2),
            'unit_vector': unit_vector_p,
            }

def to_xy_coordinates(unit_vector_angle, point):
    # returns converted unit vector coordinates in x, y coordinates
    angle_orthogonal = unit_vector_angle + math.pi / 2
    return point[0] * math.cos(unit_vector_angle) + point[1] * math.cos(angle_orthogonal), \
           point[0] * math.sin(unit_vector_angle) + point[1] * math.sin(angle_orthogonal)

def rotate_points(center_of_rotation, angle, points):
    # Requires: center_of_rotation to be a 2d vector. ex: (1.56, -23.4)
    #           angle to be in radians
    #           points to be a list or tuple of points. ex: ((1.56, -23.4), (1.56, -23.4))
    # Effects: rotates a point cloud around the center_of_rotation point by angle
    rot_points = []
    ang = []
    for pt in points:
        diff = tuple([pt[d] - center_of_rotation[d] for d in range(2)])
        diff_angle = math.atan2(diff[1], diff[0]) + angle
        ang.append(diff_angle)
        diff_length = math.sqrt(sum([d**2 for d in diff]))
        rot_points.append((center_of_rotation[0] + diff_length * math.cos(diff_angle),
                           center_of_rotation[1] + diff_length * math.sin(diff_angle)))
    return rot_points

def rectangle_corners(rectangle):
    # Requires: the output of mon_bounding_rectangle
    # Effects: returns the corner locations of the bounding rectangle
    corner_points = []
    for i1 in (.5, -.5):
        for i2 in (i1, -1 * i1):
            corner_points.append((rectangle['rectangle_center'][0] + i1 * rectangle['length_parallel'],
                            rectangle['rectangle_center'][1] + i2 * rectangle['length_orthogonal']))
    return rotate_points(rectangle['rectangle_center'], rectangle['unit_vector_angle'], corner_points)

def min_bbox(points):
    ordered_hull_points = [points[index] for index in ConvexHull(points).vertices]
    ordered_hull_points.append(ordered_hull_points[0])
    ordered_hull_points = tuple(ordered_hull_points)
    
    min_rectangle = bounding_area(0, ordered_hull_points)
    for i in range(1, len(ordered_hull_points)-1):
        rectangle = bounding_area(i, ordered_hull_points)
        if rectangle['area'] < min_rectangle['area']:
            min_rectangle = rectangle
    min_rectangle['unit_vector_angle'] = math.atan2(min_rectangle['unit_vector'][1], min_rectangle['unit_vector'][0])
    min_rectangle['rectangle_center'] = to_xy_coordinates(min_rectangle['unit_vector_angle'], min_rectangle['rectangle_center'])
    
    return np.array(rectangle_corners(min_rectangle))
    
def calculate_convex_mask(pts, img):
    
    points = tuple(map(tuple, pts[0:2,:].T))
    
    convex_hull = [points[index] for index in ConvexHull(points).vertices]
    item = (convex_hull[0][0], convex_hull[0][1])
    convex_hull.append(item)
    convex_hull = np.array(convex_hull, dtype='int32')
    
    mask = np.zeros(img.shape[0:2],dtype=np.uint8)
    
    cv2.fillPoly(mask, [convex_hull], 1)
    
    return mask.astype(np.uint8), convex_hull

In [4]:
def identify_landmarks(image, debug=False, upsamp_tol=3, det_type='68'):
    predictor = dlib.shape_predictor("shape_predictor_"+det_type+"_face_landmarks.dat")
    detector = dlib.get_frontal_face_detector()
    
    if len(image.shape) == 3:
        img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    elif len(image.shape) > 3:
        print("Incorrect dimensions for image: {0} dimensions found".format(len(img.shape[-1])))
    else:
        img = image
    
    #running with second detector
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces_alt = face_cascade.detectMultiScale(img,scaleFactor=1.3,minNeighbors=3)
    
    if debug is True: print("Found {0} ALT Faces!".format(len(faces_alt)))
    
    i = 0
    while (True):
        faces = detector(img, i)
        if len(faces) == len(faces_alt): break;
        i += 1
        if (i >= upsamp_tol):
            print("Failed to find faces. Upsample factor {0} is limit".format(upsamp_tol))
        
    pts = np.zeros((len(faces),3,int(det_type)))
    if debug is True: print("Found {0} Faces!".format(len(faces)))
    if debug is True: img_copy = np.copy(img)
    
    for i, face in enumerate(faces):
        if i+1>len(faces_alt): break;
        
        if debug is True: print(face)
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()

        landmarks = predictor(img, face)

        for n in range(0, int(det_type)):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            pts[i,0,n] = x
            pts[i,1,n] = y
            pts[i,2,n] = 1
            if debug is True: cv2.circle(img_copy, (x, y), 2, (255, 0, 0), -1)

    if debug is True: 
        print('Pts:\n', pts)
        plt.figure()
        plt.imshow(img_copy, cmap='gray')
        
    return pts

#takes a list of images as input
def multi_image_landmarks(images, detector='68'):
    pts = []
    for img in images:
        pts.append(identify_landmarks(img, det_type=detector))
    depth_counter = 0
    for i in range(len(pts)):
        depth_counter += pts[i].shape[0]
    ret_pts = np.zeros((depth_counter, 3,int(detector)))
    depth_counter = 0
    for i in range(len(pts)):
        pts_arr = pts[i]
        for k in range(pts_arr.shape[0]):
            ret_pts[depth_counter,:,:] = pts_arr[k,:,:]
            depth_counter += 1
    return ret_pts

In [5]:
def alpha_blending(frame, pts1, mask, std=4):
    
    bg = np.ones((frame.shape[0], frame.shape[1]))
    fg = np.ones((frame.shape[0], frame.shape[1]))
    
    # Face 1 box
    x1 = int(np.min(pts1[0,1,:]))
    y1 = int(np.min(pts1[0,0,:]))
    h1 = int(np.max(pts1[0,1,:]) - x1)
    w1 = int(np.max(pts1[0,0,:]) - y1)
    
    x = cv2.getGaussianKernel(int(h1), h1//std)
    y = cv2.getGaussianKernel(int(w1), w1//std)
    kernel1 = x.dot(y.T)
    kernel1 = kernel1/np.max(kernel1)
    
    # Face 2 box
    x2 = int(np.min(pts1[1,1,:]))
    y2 = int(np.min(pts1[1,0,:]))
    h2 = int(np.max(pts1[1,1,:]) - x2)
    w2 = int(np.max(pts1[1,0,:]) - y2)
    
    x = cv2.getGaussianKernel(int(h2), h2//std)
    y = cv2.getGaussianKernel(int(w2), w2//std)
    kernel2 = x.dot(y.T)    
    kernel2 = kernel2/np.max(kernel2)

#     print(x2,y2,h2,w2)
    
    bg[x1:x1+h1,y1:y1+w1] = (1-kernel1) * mask[x1:x1+h1,y1:y1+w1]
    bg[x2:x2+h2,y2:y2+w2] = (1-kernel2) * mask[x2:x2+h2,y2:y2+w2]
    bg[bg == 0] = 1
#     plt.figure()
#     plt.imshow(bg)
    
    fg[x1:x1+h1,y1:y1+w1] = kernel1 * mask[x1:x1+h1,y1:y1+w1]
    fg[x2:x2+h2,y2:y2+w2] = kernel2 * mask[x2:x2+h2,y2:y2+w2]
    fg[fg == 0] = 1

#     plt.figure()
#     plt.imshow(fg)
    
    
    return np.expand_dims(bg, axis=-1), np.expand_dims(fg, axis=-1)


# bg, fg = alpha_blending(cage, pts, mask)
# plt.figure()
# plt.imshow(bg)
# plt.figure()
# plt.imshow(fg)
# print(np.max(fg))

In [6]:
import cv2
import numpy as np
import dlib

cap = cv2.VideoCapture(0)

predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
detector = dlib.get_frontal_face_detector()

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame,1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    
    pts = np.zeros((len(faces),3,68))
    
    if len(faces) == 2:
    
        for i, face in enumerate(faces):
            x1 = face.left()
            y1 = face.top()
            x2 = face.right()
            y2 = face.bottom()
    
            landmarks = predictor(gray, face)
    
            for n in range(0, 68):
                x = landmarks.part(n).x
                y = landmarks.part(n).y
                pts[i,0,n] = x
                pts[i,1,n] = y
                pts[i,2,n] = 1
                
        H12 = computeHomography(np.squeeze(pts[0]), np.squeeze(pts[1]))
        H21 = computeHomography(np.squeeze(pts[1]), np.squeeze(pts[0]))
        
        face1 = cv2.warpPerspective(frame, H12, dsize=(frame.shape[1],frame.shape[0]))/255.0
        face2 = cv2.warpPerspective(frame, H21, dsize=(frame.shape[1],frame.shape[0]))/255.0
        
#         x1_min = int(np.min(pts[1,0,:]))
#         x1_max = int(np.max(pts[1,0,:]))
#         y1_min = int(np.min(pts[1,1,:]))
#         y1_max = int(np.max(pts[1,1,:]))
        
#         mask1 = np.zeros((frame.shape[0], frame.shape[1],1))
#         mask1[y1_min:y1_max, x1_min:x1_max,:] = 1
        
#         x2_min = int(np.min(pts[0,0,:]))
#         x2_max = int(np.max(pts[0,0,:]))
#         y2_min = int(np.min(pts[0,1,:]))
#         y2_max = int(np.max(pts[0,1,:]))
        
#         mask2 = np.zeros((frame.shape[0], frame.shape[1],1))
#         mask2[y2_min:y2_max, x2_min:x2_max,:] = 1

        mask1, _ = calculate_convex_mask(pts[1],frame)
#         plt.figure()
#         plt.imshow(mask1)
        mask2, _ = calculate_convex_mask(pts[0],frame)
#         plt.figure()
#         plt.imshow(mask2)
        
        mask = np.bitwise_or(mask1,mask2)
#         plt.figure()
#         plt.imshow(mask)
        
        bg, fg = alpha_blending(frame, pts, mask)
#         plt.figure()
#         plt.imshow(np.squeeze(bg))
#         plt.figure()
#         plt.imshow(np.squeeze(fg))
#         plt.figure()
#         plt.imshow(bg*frame/255.0)
#         plt.colorbar()

#         frame = (fg*face1/255*(mask1[:,:,None].astype(int)) + face2/255*(mask2[:,:,None].astype(int)) + bg*frame/255*(1-mask[:,:,None].astype(int)))
        
        frame = (fg*face1*(mask1[:,:,None].astype(int)) + fg*face2*(mask2[:,:,None].astype(int)) + bg*frame/255.0)*255.0
#         print(np.max(frame.astype(np.uint8)))

    cv2.imshow("Frame", frame.astype(np.uint8))
    
#     break
    
    key = cv2.waitKey(33)
    if key == 27:
        break
    
cap.release()
cv2.destroyAllWindows()

ValueError: operands could not be broadcast together with shapes (173,170) (173,0) 

In [ ]:
print(cv2.__version__)